In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE13nnn/GSE13159/matrix/GSE13159_series_matrix.txt.gz

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
# import GEOparse
import gzip
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split


#Model tesets
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier


# FAILED ATTEMPT WITH SOFT FILE

In [ ]:
#gse = GEOparse.get_GEO("GSE13159", destdir=".")
#This one to download the database and extract

# gse = GEOparse.get_GEO(filepath="GSE13159_family.soft.gz")
#This one to only extract database

In [ ]:
# # Convert samples into a pandas DataFrame
# samples = []
# for gsm_name, gsm in gse.gsms.items():
#     samples.append(gsm.table)

# # Expression matrix (genes × samples)
# # Use the GPL annotation table to map probes to genes
# expr = gse.pivot_samples('VALUE')
# print(expr.shape)  # e.g., (genes, samples)


In [ ]:
# metadata = []
# for gsm_name, gsm in gse.gsms.items():
#     md = gsm.metadata
#     metadata.append({
#         "sample_id": gsm_name,
#         "title": md["title"][0] if "title" in md else None,
#         "disease": md.get("characteristics_ch1", [""])[0]  # sometimes has disease info
#     })

# meta_df = pd.DataFrame(metadata)
# print(meta_df.head())


# USING TXT SERIES MATRIX

In [2]:
filename = "GSE13159_series_matrix.txt.gz"

with gzip.open(filename, "rt", errors="ignore") as f:
    for line in f:
        if line.startswith("!Sample"):
            print(line[:200])



!Sample_title	"MILES stage 1 data N1_0001"	"MILES stage 1 data N1_0002"	"MILES stage 1 data N1_0003"	"MILES stage 1 data N1_0004"	"MILES stage 1 data N1_0005"	"MILES stage 1 data N1_0006"	"MILES stage
!Sample_geo_accession	"GSM329407"	"GSM329408"	"GSM329409"	"GSM329410"	"GSM329411"	"GSM329412"	"GSM329413"	"GSM329414"	"GSM329415"	"GSM329416"	"GSM329417"	"GSM329418"	"GSM329419"	"GSM329420"	"GSM329421
!Sample_status	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30 2009"	"Public on Sep 30
!Sample_submission_date	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 2008"	"Oct 10 
!Sample_last_update_date	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"Nov 14 2018"	"No

In [3]:
filename = "GSE13159_series_matrix.txt.gz"

expr = pd.read_csv(filename, sep="\t", comment="!", index_col=0)
X = expr.T
print("Expression matrix shape:", X.shape)

sample_ids, labels = [], []
with gzip.open(filename, "rt", errors="ignore") as f:
    for line in f:
        if line.startswith("!Sample_geo_accession"):
            sample_ids = [s.strip('"') for s in line.strip().split("\t")[1:]]
        if line.startswith("!Sample_characteristics_ch1") and "leukemia class" in line:
            raw_labels = line.strip().split("\t")[1:]
            labels = [lbl.split(":")[-1].strip().strip('"') for lbl in raw_labels]

print("Samples in X:", len(X))
print("Samples in labels:", len(labels))

y = pd.DataFrame({"sample_id": sample_ids, "label": labels})

X.to_csv("X.csv")
y.to_csv("y.csv", index=False)

print("✅ Saved X.csv and y.csv")
print("Unique classes:", y['label'].unique())

Expression matrix shape: (2096, 54675)
Samples in X: 2096
Samples in labels: 2096
✅ Saved X.csv and y.csv
Unique classes: ['mature B-ALL with t(8;14)' 'Pro-B-ALL with t(11q23)/MLL'
 'c-ALL/Pre-B-ALL with t(9;22)' 'T-ALL' 'ALL with t(12;21)'
 'ALL with t(1;19)' 'ALL with hyperdiploid karyotype'
 'c-ALL/Pre-B-ALL without t(9;22)' 'AML with t(8;21)' 'AML with t(15;17)'
 'AML with inv(16)/t(16;16)' 'AML with t(11q23)/MLL'
 'AML with normal karyotype + other abnormalities'
 'AML complex aberrant karyotype' 'CLL' 'CML' 'MDS'
 'Non-leukemia and healthy bone marrow']


In [4]:
X = pd.read_csv("X.csv", index_col=0)
y = pd.read_csv("y.csv", index_col=0).squeeze()

In [5]:
print("Features shape:", X.shape)   # (2096, 54675)
print("Labels shape:", y.shape)     # (2096,)
print("Unique labels:", y.unique())


Features shape: (2096, 54675)
Labels shape: (2096,)
Unique labels: ['mature B-ALL with t(8;14)' 'Pro-B-ALL with t(11q23)/MLL'
 'c-ALL/Pre-B-ALL with t(9;22)' 'T-ALL' 'ALL with t(12;21)'
 'ALL with t(1;19)' 'ALL with hyperdiploid karyotype'
 'c-ALL/Pre-B-ALL without t(9;22)' 'AML with t(8;21)' 'AML with t(15;17)'
 'AML with inv(16)/t(16;16)' 'AML with t(11q23)/MLL'
 'AML with normal karyotype + other abnormalities'
 'AML complex aberrant karyotype' 'CLL' 'CML' 'MDS'
 'Non-leukemia and healthy bone marrow']


In [6]:
# Clean label text
y = y.str.replace("leukemia class: ", "", regex=False)
y = y.str.strip()
print(y.value_counts())


label
CLL                                                448
AML with normal karyotype + other abnormalities    351
c-ALL/Pre-B-ALL without t(9;22)                    237
MDS                                                206
T-ALL                                              174
c-ALL/Pre-B-ALL with t(9;22)                       122
CML                                                 76
Non-leukemia and healthy bone marrow                74
Pro-B-ALL with t(11q23)/MLL                         70
ALL with t(12;21)                                   58
AML complex aberrant karyotype                      48
ALL with hyperdiploid karyotype                     40
AML with t(8;21)                                    40
AML with t(11q23)/MLL                               38
AML with t(15;17)                                   37
ALL with t(1;19)                                    36
AML with inv(16)/t(16;16)                           28
mature B-ALL with t(8;14)                           13
Name

In [7]:

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(dict(zip(le.classes_, range(len(le.classes_)))))

{'ALL with hyperdiploid karyotype': 0, 'ALL with t(12;21)': 1, 'ALL with t(1;19)': 2, 'AML complex aberrant karyotype': 3, 'AML with inv(16)/t(16;16)': 4, 'AML with normal karyotype + other abnormalities': 5, 'AML with t(11q23)/MLL': 6, 'AML with t(15;17)': 7, 'AML with t(8;21)': 8, 'CLL': 9, 'CML': 10, 'MDS': 11, 'Non-leukemia and healthy bone marrow': 12, 'Pro-B-ALL with t(11q23)/MLL': 13, 'T-ALL': 14, 'c-ALL/Pre-B-ALL with t(9;22)': 15, 'c-ALL/Pre-B-ALL without t(9;22)': 16, 'mature B-ALL with t(8;14)': 17}


In [8]:
print("NaNs ", np.isnan(X.values).sum())
print("Infs ", np.isinf(X.values).sum())
X = X.fillna(0)


NaNs  94320
Infs  0


In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(X_scaled.shape)

(2096, 54675)


In [10]:
selector = SelectKBest(score_func=f_classif, k=10000)
X_reduced = selector.fit_transform(X_scaled, y_encoded)

print("Reduced shape:", X_reduced.shape)


Reduced shape: (2096, 10000)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [54613 54614 54615 54616 54617 54618 54619 54620 54621 54622 54623 54624
 54639 54640 54641 54645 54646 54647 54648 54649 54650 54651 54652 54653
 54654 54655 54656 54657 54658 54659 54660 54661 54662 54663 54664 54665
 54666 54667 54668 54669 54670 54671 54672 54673 54674] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [11]:

X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y_encoded, test_size=0.2, random_state=69, stratify=y_encoded
)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(1676, 10000) (1676,) (420, 10000) (420,)


In [12]:
models = {
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM (RBF)": SVC(kernel="rbf", probability=True),
    "Logistic Regression": LogisticRegression(max_iter=500, solver="liblinear"),
    "Neural Net (MLP)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
}

#NOT USING GRADIENT BOOSTING AS IT WAS TAKING TOO MUCH TIME

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {}
for name, model in models.items():
    scores = cross_val_score(model, X_reduced, y_encoded, cv=cv, scoring="accuracy")
    results[name] = (np.mean(scores), np.std(scores))
    print(f"{name:20s}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")

Naive Bayes         : 0.816 ± 0.018
Random Forest       : 0.863 ± 0.010
KNN                 : 0.836 ± 0.008
SVM (RBF)           : 0.879 ± 0.007
Logistic Regression : 0.913 ± 0.012
Neural Net (MLP)    : 0.904 ± 0.004
